In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import pandas as pd

data = pd.read_csv('/kaggle/input/fake-news/train.csv')


In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.nunique()

id        20800
title     19803
author     4201
text      20386
label         2
dtype: int64

In [5]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
data=data.dropna()

In [7]:
data.reset_index(inplace=True)

In [8]:
data.isnull().sum()

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
X = data['title']
y = data['label']

In [10]:
X.isnull().sum()

0

In [11]:
X.shape

(18285,)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

2024-05-15 15:31:45.605135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 15:31:45.605266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 15:31:45.759013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Check the tensorflow version**

In [14]:
tf.__version__

'2.15.0'

# **Import libraries to create neural network with embedding layer**

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import nltk

In [16]:
voc_size=5000

In [17]:
headings = X.copy()

In [18]:
headings.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [19]:
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Text Preprocessing Phase**

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]

for i in range(0,len(headings)):
    heading = re.sub('[^a-zA-Z]',' ',headings[i])
    heading = heading.lower()
    heading = str(heading).split()
    
    heading = [ps.stem(word) for word in heading if not word in stopwords.words('english')]
    heading = ' '.join(heading)
    corpus.append(heading)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

# **Convert the words into one hot representation**

In [23]:
one_hot_reprt = [one_hot(word, voc_size) for word in corpus]

In [24]:
one_hot_reprt

[[4061, 203, 4867, 853, 3341, 3203, 2618, 3924, 2221, 452],
 [4610, 3488, 3049, 313, 1726, 989, 1950],
 [3485, 4855, 4215, 3694],
 [3538, 2281, 451, 2085, 3906, 1902],
 [1295, 1726, 1646, 2899, 4908, 1441, 1726, 1916, 4085, 3883],
 [4728,
  4679,
  810,
  1814,
  3135,
  398,
  890,
  3545,
  884,
  1575,
  525,
  3663,
  1180,
  2779,
  1950],
 [447, 2811, 3952, 2570, 749, 2037, 3840, 2198, 4732, 2760, 4613],
 [1381, 4923, 3096, 1574, 3442, 3141, 398, 3914, 4732, 2760, 4613],
 [252, 4648, 107, 1134, 4775, 684, 299, 4483, 398, 3192],
 [1201, 505, 1770, 724, 1170, 894, 458, 893],
 [3307, 1048, 1751, 4042, 2962, 2737, 1886, 4276, 2297, 219, 4076],
 [2085, 4624, 3341, 684, 398, 3442],
 [4567, 3862, 1950, 2540, 936, 3501, 4869, 1946, 914],
 [1400, 4642, 1973, 728, 3705, 1464, 241, 4732, 2760, 4613],
 [3928, 452, 2576, 1603, 3325, 4732, 2760, 4613],
 [221, 123, 1692, 3808, 4859, 1602, 4493, 247, 2558, 1826],
 [4875, 282, 3488],
 [1243, 1859, 4181, 4345, 398, 4990, 3362, 1950],
 [4619, 14, 3

In [25]:
sent_length=50
embedded_docs=pad_sequences(one_hot_reprt,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3924 2221  452]
 [   0    0    0 ... 1726  989 1950]
 [   0    0    0 ... 4855 4215 3694]
 ...
 [   0    0    0 ... 4732 2760 4613]
 [   0    0    0 ...  579 3485  191]
 [   0    0    0 ... 2338 3231 4682]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 4061,  203, 4867,  853,
       3341, 3203, 2618, 3924, 2221,  452], dtype=int32)

In [27]:
embedded_vector_features=50
model=Sequential()
model.add(Embedding(voc_size, embedded_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np

x_final=np.array(embedded_docs)
y_final = np.array(y)

In [30]:
x_final

array([[   0,    0,    0, ..., 3924, 2221,  452],
       [   0,    0,    0, ..., 1726,  989, 1950],
       [   0,    0,    0, ..., 4855, 4215, 3694],
       ...,
       [   0,    0,    0, ..., 4732, 2760, 4613],
       [   0,    0,    0, ...,  579, 3485,  191],
       [   0,    0,    0, ..., 2338, 3231, 4682]], dtype=int32)

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_final,y_final, test_size=0.30,random_state=42)

In [32]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.8287 - loss: 0.3596 - val_accuracy: 0.9172 - val_loss: 0.1984
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.9527 - loss: 0.1223 - val_accuracy: 0.9209 - val_loss: 0.1963
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9757 - loss: 0.0720 - val_accuracy: 0.9169 - val_loss: 0.2355
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.9875 - loss: 0.0402 - val_accuracy: 0.9134 - val_loss: 0.2997
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9933 - loss: 0.0253 - val_accuracy: 0.9032 - val_loss: 0.3705
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.9950 - loss: 0.0177 - val_accuracy: 0.9063 - val_loss: 0.4393
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.9973 - loss: 0.0105 - val_accuracy: 0.9039 - val_loss: 0.5235
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.9984 - loss: 0.0061 - 

In [33]:
y_pred=model.predict(x_test)

172/172 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step


In [34]:
y_pred=[1 if i>=0.5 else 0 for i in y_pred ]

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2807,  300],
       [ 216, 2163]])

In [36]:
accuracy_score(y_test,y_pred)

0.9059423988333941